配置镜像地址

In [1]:
import os
# 设置hf-mirror镜像地址
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# 查看是否成功结果
hf_endpoint = os.getenv('HF_ENDPOINT')
print('HF_ENDPOINT:', hf_endpoint)

HF_ENDPOINT: https://hf-mirror.com


登录hugging face

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import sys
sys.path.append("/home/pod/shared-nvme/ChineseBERT/ChineseBertSourceCode")
sys.path

['/home/pod/miniconda3/lib/python38.zip',
 '/home/pod/miniconda3/lib/python3.8',
 '/home/pod/miniconda3/lib/python3.8/lib-dynload',
 '',
 '/home/pod/miniconda3/lib/python3.8/site-packages',
 '/home/pod/shared-nvme/ChineseBERT/ChineseBertSourceCode']

处理tsv数据，构建hugging face数据

In [3]:
from datasets import load_dataset
data_files = {"train":"./data/train.tsv", "test":"./data/test.tsv"}
ChnSetiCorp_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
ChnSetiCorp_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text_a'],
        num_rows: 9146
    })
    test: Dataset({
        features: ['label', 'text_a'],
        num_rows: 1200
    })
})

In [11]:
ChnSetiCorp_dataset["train"][0]

{'label': 1,
 'text_a': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般'}

加载ChineseBert模型<NEWLINE> 并且用于文本分类任务

In [4]:
from transformers import AutoTokenizer, AutoModel
from datasetsProcess.chn_senti_corp_dataset import ChnSentCorpDataset
from models.modeling_glycebert import GlyceBertForSequenceClassification
from datasetsProcess.bert_dataset import BertDataset
data_path = "/home/pod/shared-nvme/NLP-study/文本分类/b站弹幕情感分析/data/train.tsv"
model_checkpoint = "/home/pod/shared-nvme/ChineseBERT/models--ShannonAI--ChineseBERT-base"
train_data = ChnSentCorpDataset(chinese_bert_path = model_checkpoint, data_path = data_path)
model = GlyceBertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
tokenizer = BertDataset(model_checkpoint)

Some weights of GlyceBertForSequenceClassification were not initialized from the model checkpoint at /home/pod/shared-nvme/ChineseBERT/models--ShannonAI--ChineseBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model

GlyceBertForSequenceClassification(
  (bert): GlyceBertModel(
    (embeddings): FusionBertEmbeddings(
      (word_embeddings): Embedding(23236, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (pinyin_embeddings): PinyinEmbedding(
        (embedding): Embedding(32, 128)
        (conv): Conv1d(128, 768, kernel_size=(2,), stride=(1,))
      )
      (glyph_embeddings): GlyphEmbedding(
        (embedding): Embedding(23236, 1728)
      )
      (glyph_map): Linear(in_features=1728, out_features=768, bias=True)
      (map_fc): Linear(in_features=2304, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_

In [38]:
from torch.utils.data import DataLoader
from functools import partial
from datasetsProcess.collate_functions import collate_to_max_length
dataloader = DataLoader(
        dataset=train_data,
        batch_size=20,
        num_workers=8,
        shuffle=False,
        collate_fn=partial(collate_to_max_length, fill_values=[0, 0, 0]),
    )
for input_ids, pinyin_ids, label in dataloader:
    bs, length = input_ids.shape
    print(input_ids.shape)
    print(pinyin_ids.reshape(bs, length, -1).shape)
    print(label.shape)
    print()
next(iter(dataloader))

torch.Size([20, 163])
torch.Size([20, 163, 8])
torch.Size([20, 1])

torch.Size([20, 365])
torch.Size([20, 365, 8])
torch.Size([20, 1])

torch.Size([20, 207])
torch.Size([20, 207, 8])
torch.Size([20, 1])

torch.Size([20, 185])
torch.Size([20, 185, 8])
torch.Size([20, 1])

torch.Size([20, 480])
torch.Size([20, 480, 8])
torch.Size([20, 1])

torch.Size([20, 199])
torch.Size([20, 199, 8])
torch.Size([20, 1])

torch.Size([20, 503])
torch.Size([20, 503, 8])
torch.Size([20, 1])

torch.Size([20, 465])
torch.Size([20, 465, 8])
torch.Size([20, 1])

torch.Size([20, 338])
torch.Size([20, 338, 8])
torch.Size([20, 1])

torch.Size([20, 448])
torch.Size([20, 448, 8])
torch.Size([20, 1])

torch.Size([20, 389])
torch.Size([20, 389, 8])
torch.Size([20, 1])

torch.Size([20, 512])
torch.Size([20, 512, 8])
torch.Size([20, 1])

torch.Size([20, 209])
torch.Size([20, 209, 8])
torch.Size([20, 1])

torch.Size([20, 507])
torch.Size([20, 507, 8])
torch.Size([20, 1])

torch.Size([20, 509])
torch.Size([20, 509, 8])
t

[tensor([[ 101, 6848, 2885,  ...,    0,    0,    0],
         [ 101, 8115,  119,  ...,    0,    0,    0],
         [ 101, 2791, 7313,  ...,    0,    0,    0],
         ...,
         [ 101, 2523, 1962,  ...,    0,    0,    0],
         [ 101, 4023,  778,  ...,    0,    0,    0],
         [ 101, 1079, 2100,  ...,    0,    0,    0]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 tensor([[1],
         [1],
         [0],
         [0],
         [1],
         [0],
         [0],
         [0],
         [1],
         [1],
         [0],
         [1],
         [1],
         [1],
         [0],
         [0],
         [1],
         [0],
         [1],
         [0]])]

In [20]:
import torch
input_ids, pinyin_ids = tokenizer.tokenize_sentence("李洋是个臭傻逼，纯贱种")
length = input_ids.shape[0]
input_ids = input_ids.view(1, length)
pinyin_ids = pinyin_ids.view(1, length, 8)
attention_mask = torch.ones(1, length)

In [21]:
out = model.forward(input_ids, pinyin_ids, attention_mask)

/home/pod/miniconda3/lib/python3.8/site-packages/transformers/modeling_utils.py:1161: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [25]:
from torch.nn.functional import softmax
softmax(out.logits)

/tmp/ipykernel_2435/1071611802.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax(out.logits)


tensor([[0.4955, 0.5045]], grad_fn=<SoftmaxBackward0>)

In [5]:
import evaluate
metric = evaluate.combine(["accuracy", "f1", "precision", "recall", "matthews_correlation"])

In [6]:
from transformers import TrainingArguments, Trainer
model_name = "ChineseBert"
task = "sentiment"
batch_size = 32
metric_name = "matthews_correlation"
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    report_to="none"
    # push_to_hub=True,
)

In [7]:
new_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone-chinese")

In [35]:
new_tokenizer(ChnSetiCorp_dataset["train"][0]["text_a"])

{'input_ids': [101, 6848, 2885, 4403, 3736, 5709, 1736, 4638, 1333, 1728, 2218, 3221, 3175, 912, 8024, 3300, 4510, 1220, 2820, 3461, 4684, 2970, 1168, 6809, 3862, 6804, 8024, 1453, 1741, 7623, 7667, 510, 7608, 2443, 510, 1555, 1767, 510, 6631, 2356, 510, 3033, 855, 671, 2418, 936, 1059, 511, 6983, 2421, 6163, 934, 671, 5663, 8024, 852, 6820, 5050, 3146, 3815, 511, 3807, 3737, 1762, 1920, 1828, 4638, 2238, 7553, 8024, 1728, 3634, 2523, 2207, 8024, 679, 6814, 1957, 1036, 948, 3221, 1599, 3614, 511, 1259, 4638, 3193, 7623, 3221, 6205, 2466, 4638, 8024, 6820, 5050, 705, 2168, 511, 3302, 1218, 1408, 8024, 671, 5663, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 

In [8]:
def preprocess_function(example):
    return new_tokenizer(example["text_a"], truncation=True, max_length=128)

In [9]:
encoded_dataset = ChnSetiCorp_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9146 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [10]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text_a', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9146
    })
    test: Dataset({
        features: ['label', 'text_a', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
})

In [11]:
task = "sentiment_test"
batch_size = 32

In [12]:
from transformers import TrainingArguments, Trainer
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    report_to="none"
    # push_to_hub=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone-chinese", num_labels=2, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-tone-chinese and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=new_tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_3761/1332936924.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Matthews Correlation
1,No log,0.230417,0.932500,0.932218,0.948893,0.916118,0.865569
2,0.136600,0.230611,0.939167,0.938397,0.963605,0.914474,0.879569
3,0.136600,0.211090,0.946667,0.946578,0.961017,0.932566,0.893759
4,0.051600,0.236038,0.950000,0.949917,0.964407,0.935855,0.900427
5,0.051600,0.252850,0.950833,0.951280,0.955224,0.947368,0.901691


TrainOutput(global_step=1430, training_loss=0.0731728607124382, metrics={'train_runtime': 239.9295, 'train_samples_per_second': 190.598, 'train_steps_per_second': 5.96, 'total_flos': 3609620568483840.0, 'train_loss': 0.0731728607124382, 'epoch': 5.0})

In [25]:
trainer.evaluate()

{'eval_loss': 0.2528499960899353,
 'eval_accuracy': 0.9508333333333333,
 'eval_f1': 0.9512799339388934,
 'eval_precision': 0.9552238805970149,
 'eval_recall': 0.9473684210526315,
 'eval_matthews_correlation': 0.9016914241226822,
 'eval_runtime': 2.0123,
 'eval_samples_per_second': 596.322,
 'eval_steps_per_second': 18.884,
 'epoch': 5.0}